In [2]:
# Run the pip install command below if you don't already have the library
# !pip install git+https://github.com/JustAnotherArchivist/snscrape.git

# Imports
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import csv



In [25]:
def getQueryLists(file_name, hashtags=None,dates=" until:2021-04-21 since:2010-01-01 ", min_faves=10, min_retweets=10):
# returns a list of ca. 60 queries for the given hastags, dates and minimum virality which
# in total cover queries for all UK MPs for these criteria
# hastags format: "(#covid OR #easter)"
# dates in the format: " until:2021-03-01 since:2021-01-01 "

    mpSearches=getMPSearchStringList(file_name=file_name)
    allQueries=[]
    for i, mpSearch in enumerate(mpSearches) :
        if hashtags != None:
            allQueries.append(hashtags + mpSearch + dates + " min_faves:" + str(min_faves) + " min_retweets:" + str(min_retweets))
        else:
            allQueries.append(mpSearch + dates + " min_faves:" + str(min_faves) + " min_retweets:" + str(min_retweets))
        
    return allQueries


In [26]:
def getChunks(file_name):
    # Derived from the csv of all MPs from https://www.politics-social.com/list/followers
    # a list of lists of twitter handles for all UK MPs is returned
    
    a_file = open(file_name, "r")
    list_of_lists = []

    for line in a_file:
        stripped_line = line.strip()
        list_of_lists.append(stripped_line)

        splitlists=np.array_split(np.array(list_of_lists), 60) 
        # splitlists=np.array_split(np.array(list_of_lists), 10) 

    a_file.close()
    return splitlists


In [20]:
def getMPSearchStringList(file_name = "C:/Users/kbutl/OneDrive/janice/github/TwitterScraper/snscrape/python-wrapper/serious_news.txt"):
# returns a list of twitter query strings of the form 
# (from:zarahsultana OR from:Y_FovargueMP OR from:YvetteCooperMP OR from:YasminQureshiMP
# the queries are limited in length due to Twitter restrictions, so a list of the queries
# is needed. ca. 600 MPs --> list of ca. 60 queries

    chunks=getChunks(file_name = file_name)
    mp_search_strings = []
    for i, chk in enumerate(chunks):
        mp_search_strings.append("(from:")
        for j, mp in enumerate(chk):
            if j!=0:
                mp_search_strings[i] = mp_search_strings[i] + " OR from:" + mp
            else:
                mp_search_strings[i] = mp_search_strings[i] + mp
        mp_search_strings[i] = mp_search_strings[i] +")"

    return mp_search_strings


In [27]:
# Setting variables to be used for scraping below
maxTweets = 250000          # set as required
file_name = "mps.txt"       # set as required (see options below)

# Using TwitterSearchScraper to scrape data and append tweets to list
#  hashtags=None,dates=" until:2021-03-01 since:2021-01-01 ", min_faves=10, min_retweets=10
# file_name = "journalists.txt"):
# file_name = "serious_news.txt"
# file_name = "comedians.txt"

# Creating list to append tweet data to
tweets_list1 = []

ql=getQueryLists(file_name)

for mpQs in getQueryLists(file_name):
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(mpQs).get_items()):
        if i>maxTweets:
            break
        tweets_list1.append([str(tweet.id), tweet.content, tweet.user.username,tweet.user.followersCount, str(tweet.conversationId),        tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount],  )

In [ ]:
# Get some serious tweets, set the label to 0 (no humour)
# Setting variables to be used below
maxTweets = 200000

# Creating list to append tweet data to
tweets_list0 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
#  hashtags=None,dates=" until:2021-03-01 since:2021-01-01 ", min_faves=10, min_retweets=10
file_name = "C:/Users/kbutl/OneDrive/janice/github/TwitterScraper/snscrape/python-wrapper/serious_news.txt"

for mpQs in getQueryLists(file_name, min_faves=0,min_retweets=0):
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(mpQs).get_items()):
        if i>maxTweets:
            break
        tweets_list0.append(["0", tweet.content],  )


In [ ]:
# Creating a dataframe from the serious tweets list above
tweets_df0 = pd.DataFrame(tweets_list0, columns=['label', 'sentence'])
# Display first 5 entries from dataframe
tweets_df0.head()

In [28]:
# Creating a dataframe from the tweets list above
tweets_df1 = pd.DataFrame(tweets_list1, columns=['tweetId', 'content', 'username','followers', 'conversationId', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount'])

# Display first 5 entries from dataframe
tweets_df1.head()

,tweetId,content,username,followers,conversationId,replyCount,retweetCount,likeCount,quoteCount
0,1384617849415688192,American police forces were established to cat...,robdelaney,1581288,1384617849415688192,78,455,4774,10
1,1384613550526803969,"Oh my god, I sound so horny as a French person...",WeeMissBea,508964,1384613550526803969,55,21,1016,1
2,1384597084448464906,Take down immediatejt https://t.co/CiVIQEfo1V,robdelaney,1581288,1384597084448464906,17,46,1702,1
3,1384498297381945346,https://t.co/6jKt6gC87Z,robdelaney,1581288,1384498297381945346,34,32,623,5
4,1384482731464138758,Susan's #CatofTheDay is Happ.\n\nThank you @Fr...,SusanCalman,260039,1384482731464138758,15,24,1138,1


In [29]:
# Export dataframe into a CSV
# tweets_df1.to_csv('user-tweets.csv', sep='\t', index=False, encoding="utf-16")

tweets_df1.to_csv('comedian-tweets250.csv', sep='\t', index=False, encoding="utf-16")
# tweets_df1.to_csv('user-tweets.csv', sep='\t', index=False, encoding="utf-8-sig")
# tweets_df1.to_excel('xuser-tweets.xlsx',  index=False)



In [ ]:
# clean up data
tweets_df0["sentence"] = tweets_df0["sentence"].replace(r'\n',' ',regex=True)
tweets_df0["sentence"] = tweets_df0["sentence"].replace(r'\t',' ',regex=True)
tweets_df0["sentence"] = tweets_df0["sentence"].replace(r'\r',' ',regex=True)
tweets_df0 = tweets_df0.dropna()

# tweets_df0.to_csv('serious_tweets.tsv', sep='\t', quoting=csv.QUOTE_NONE, escapechar='\\', index=None, header=['label','sentence']
tweets_df0.to_csv('serious_tweets.tsv', sep='\t', quoting=csv.QUOTE_NONE, escapechar='\\', index=None, header=None,encoding="UTF-8")



In [ ]:
# Import dataframe from csv
import pandas as pd
tweet_file="user-tweets.csv"
columns_in=['tweetId', 'content', 'username','followers', 'conversationId', 
         'replyCount', 'retweetCount', 'likeCount', 'quoteCount']
tweet_df = pd.read_csv(tweet_file, delimiter='\t', header=None, names=columns_in, lineterminator='\n')

# Query by Text Search
The code below will scrape for 500 tweets between June 1st, 2020 and July 31st, 2020, by a text search then provide a CSV file with Pandas

In [ ]:
# Setting variables to be used below
maxTweets = 50000

# Creating list to append tweet data to
tweets_list2 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('its the elephant since:2020-06-01 until:2020-07-31').get_items()):
    if i>maxTweets:
        break
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username])

In [ ]:
# Creating a dataframe from the tweets list above
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

# Display first 5 entries from dataframe
tweets_df2.head()

In [ ]:
# Export dataframe into a CSV
tweets_df1.to_csv('text-query-tweets.csv', sep='\t', index=False)

In [ ]:
# !pip install mayavi
import networkx as nx

from mayavi import mlab

H = nx.from_pandas_edgelist(tweets_df1, 'Tweet Id','conversationId') 
# reorder nodes from 0,len(G)-1
G = nx.convert_node_labels_to_integers(H)
# 3d spring layout
pos = nx.spring_layout(G, dim=3)
# pos = nx.random_layout(G, dim=3)
# numpy array of x,y,z positions in sorted node order
xyz = np.array([pos[v] for v in sorted(G)])
# scalar colors
scalars = np.array(list(G.nodes())) + 5

pts = mlab.points3d(
    xyz[:, 0],
    xyz[:, 1],
    xyz[:, 2],
    scalars,
    scale_factor=0.1,
    scale_mode="none",
    colormap="Blues",
    resolution=20,
)

pts.mlab_source.dataset.lines = np.array(list(G.edges()))
tube = mlab.pipeline.tube(pts, tube_radius=0.01)
mlab.pipeline.surface(tube, color=(0.8, 0.8, 0.8))
# mlab.init_notebook()
mlab.show()

In [ ]:
# reread tweetfile

dtyps={'tweetId':str, 'content':str, 'username':str,'followers':int, 'conversationId':str, 
         'replyCount':int, 'retweetCount':int, 'likeCount':int, 'quoteCount':int}

tweet_df = pd.read_csv('user-tweets.csv', delimiter='\t', header=None, dtype=dtyps,
                        lineterminator='\n',encoding='utf-16')


In [ ]:
# C:\Users\kbutl\OneDrive\janice\github\rJokes\rJokesData\data\tweets_w_humour.tsv
# rJokes\rJokesData\data\tweets_w_humour.tsv
# C:\Users\kbutl\OneDrive\janice\github\TwitterScraper\snscrape\python-wrapper\user-tweets.csv
# C:\Users\kbutl\OneDrive\janice\github\rJokes\rJokesData\data\tweets_w_humour_korr.tsv

dtyps={'tweetId':str, 'content':str, 'username':str,'followers':int, 'conversationId':str, 
         'replyCount':int, 'retweetCount':int, 'likeCount':int, 'quoteCount':int, 'Humour':str}

# tw_df_humour = pd.read_csv('../../../TwitterScraper/snscrape/python-wrapper/tweets_w_humour_korr.tsv', delimiter='\t', header=None, dtype=dtyps,
#                         lineterminator='\n',encoding='utf-16')
tw_df_journalists = pd.read_csv('../../../TwitterScraper/snscrape/python-wrapper/jtweets_w_humour_distilbert_balanced.tsv', delimiter='\t', dtype=dtyps,lineterminator='\n',encoding='utf-16')



In [ ]:
dtyps={'tweetId':str, 'content':str, 'username':str,'followers':int, 'conversationId':str, 
         'replyCount':int, 'retweetCount':int, 'likeCount':int, 'quoteCount':int, 'Humour':str}

# tw_df_humour = pd.read_csv('../../../rJokes/rJokesData/data/tweets_w_humour_korr.tsv', delimiter='\t', header=None, dtype=dtyps,
#                         lineterminator='\n',encoding='utf-16')
tw_df_humourdistilbert120k = pd.read_csv('../../../rJokes/rJokesData/data/tweets_w_humour_distilbert120k.tsv', delimiter='\t', dtype=dtyps,lineterminator='\n',encoding='utf-16')

In [6]:
# Wrangle the midnight files (HUMOUR Type) into shape for training, adding serious stuff in equal measure
from sklearn.model_selection import train_test_split

df_midnight = pd.read_csv('../../../rJokes/rJokesData/data_annot/annotatedX.tsv', sep='\t', header=None,names=['sentence','label0','labela','label'],encoding='utf-8')

df_midnight.pop(df_midnight.columns[2])     # remove humour degree column
df_midnight.pop(df_midnight.columns[1])     # remove original midnight scores (0,1,2)
df_midnight = df_midnight.dropna()


df_satire = pd.read_csv('../../../rJokes/rJokesData/data_annot/reddit_satire.tsv', sep='\t', header=None,names=['label','sentence','label0'],encoding='utf-8')
df_satire.pop(df_satire.columns[0])     # remove humour degree column
df_satire = df_satire.dropna()

# read in non-humourous data
file_in='../../../TwitterScraper/snscrape/python-wrapper/serious_tweets.tsv'
serious = pd.read_csv(file_in, sep='\t', header=None, names=['label','sentence'], encoding="UTF-8")
# serious = pd.read_csv(file_in, delimiter='\t', quoting=csv.QUOTE_NONE, escapechar='\\', header=None, names=['label','sentence'], encoding="UTF-8",lineterminator='\n')
serious = serious.replace([np.inf, -np.inf], np.nan)
serious = serious.dropna()
columns_titles = ["sentence","label"]

serious=serious.reindex(columns=columns_titles)
df_satire=df_satire.reindex(columns=columns_titles)

df_midnight =pd.concat([df_midnight, df_satire])                                            # add in the satire
df_midnight =pd.concat([df_midnight, serious[:min(int(len(df_midnight)),len(serious))]])    # add in the serious

# clean up where data's missing or wrong due to concatenations
df_midnight = df_midnight.dropna()

# clean up special characters
df_midnight["sentence"] = df_midnight["sentence"].replace(r'\n',' ',regex=True)
df_midnight["sentence"] = df_midnight["sentence"].replace(r'\t',' ',regex=True)
df_midnight["sentence"] = df_midnight["sentence"].replace(r'\r',' ',regex=True)
df_midnight["label"] = df_midnight["label"].replace(r'\r','',regex=True)
df_midnight["label"] = df_midnight["label"].replace(r' ','',regex=True)

df_midnight = df_midnight[df_midnight['sentence'] != 'sentence']    #remove all rows containing "sentence"
assert df_midnight["sentence"].isnull().any() == False, "there were NaNs in the sentence column, ERROR"
assert df_midnight["label"].isnull().any() == False, "there were NaNs in the score column, ERROR"

num_degrees=8   # Humour type has 0-8 possibilities
subdir_name='../../TwitterScraper/snscrape/python-wrapper'

train, dev = train_test_split(df_midnight, test_size=0.2, random_state=42)
dev, test = train_test_split(dev, test_size=0.5, random_state=42)
print("writing train, dev, and test with shapes", train.shape, dev.shape, test.shape)
suffix="_htype_0to" + str(num_degrees)
train.to_csv(os.path.join("..", subdir_name, "train"+suffix+".tsv"), sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")
dev.to_csv(os.path.join("..", subdir_name, "dev"+suffix+".tsv"),  sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")
test.to_csv(os.path.join("..", subdir_name, "test"+suffix+".tsv"),  sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")

writing train, dev, and test with shapes (18340, 2) (2292, 2) (2293, 2)


In [7]:
# Wrangle the midnight files (HUMOUR degree) into shape for training, adding serious stuff in equal measure
from sklearn.model_selection import train_test_split

df_midnight = pd.read_csv('../../../rJokes/rJokesData/data_annot/annotatedX.tsv', sep='\t', header=None,names=['sentence','label0','labela','label'],encoding='utf-8')

df_midnight.pop(df_midnight.columns[1])     # remove original midnight scores (0,1,2)
df_midnight.pop(df_midnight.columns[2])     # remove humour degree column
df_midnight = df_midnight.dropna()

columns_titles = ["sentence","label"]

df_satire = pd.read_csv('../../../rJokes/rJokesData/data_annot/reddit_satire.tsv', sep='\t', header=None,names=['label','sentence','label0'],encoding='utf-8')
df_satire.pop(df_satire.columns[2])     # remove humour degree column
df_satire = df_satire.dropna()

# read in non-humourous data
file_in='../../../TwitterScraper/snscrape/python-wrapper/serious_tweets.tsv'
serious = pd.read_csv(file_in, sep='\t', header=None, names=['label','sentence'], encoding="UTF-8")
# serious = pd.read_csv(file_in, delimiter='\t', quoting=csv.QUOTE_NONE, escapechar='\\', header=None, names=['label','sentence'], encoding="UTF-8",lineterminator='\n')
serious = serious.replace([np.inf, -np.inf], np.nan)
serious = serious.dropna()

serious=serious.reindex(columns=columns_titles) # Swap order of columns
df_satire=df_satire.reindex(columns=columns_titles)
df_midnight.columns=columns_titles # correct column names

df_midnight =pd.concat([df_midnight, df_satire])                                            # add in the satire
df_midnight =pd.concat([df_midnight, serious[:min(int(len(df_midnight)),len(serious))]])    # add in the serious

# clean up where data's missing or wrong due to concatenations
df_midnight = df_midnight.dropna()

# clean up special characters
df_midnight["sentence"] = df_midnight["sentence"].replace(r'\n',' ',regex=True)
df_midnight["sentence"] = df_midnight["sentence"].replace(r'\t',' ',regex=True)
df_midnight["sentence"] = df_midnight["sentence"].replace(r'\r',' ',regex=True)
df_midnight["label"] = df_midnight["label"].replace(r'\r','',regex=True)
df_midnight["label"] = df_midnight["label"].replace(r' ','',regex=True)

df_midnight = df_midnight[df_midnight['sentence'] != 'sentence']    #remove all rows containing "sentence"
assert df_midnight["sentence"].isnull().any() == False, "there were NaNs in the sentence column, ERROR"
assert df_midnight["label"].isnull().any() == False, "there were NaNs in the score column, ERROR"

num_degrees=5   # Humour degree has 0-5 possibilities
subdir_name='../../TwitterScraper/snscrape/python-wrapper'

train, dev = train_test_split(df_midnight, test_size=0.2, random_state=42)
dev, test = train_test_split(dev, test_size=0.5, random_state=42)
print("writing train, dev, and test with shapes", train.shape, dev.shape, test.shape)
suffix="_hdeg_0to" + str(num_degrees)
train.to_csv(os.path.join("..", subdir_name, "train"+suffix+".tsv"), sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")
dev.to_csv(os.path.join("..", subdir_name, "dev"+suffix+".tsv"),  sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")
test.to_csv(os.path.join("..", subdir_name, "test"+suffix+".tsv"),  sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")

writing train, dev, and test with shapes (19983, 2) (2498, 2) (2498, 2)


In [10]:
# Just split give tsv's into train, tes, dev
from sklearn.model_selection import train_test_split

dtyps={'sentence':str, 'label':str}

# tweet_df = pd.read_csv(tweet_file, delimiter='\t', header=None, dtype=dtyps,lineterminator='\n',encoding='utf-16')

df_deg = pd.read_csv('../../data-training/complete/htypes.tsv', sep='\t', dtype=dtyps,encoding='utf-8')
# df_deg = pd.read_csv('../../data-training/complete/hdeg.tsv', sep='\t', header=None,names=['sentence','label'],encoding='utf-8')

# columns_titles = ["sentence","label"]

df_deg = df_deg.dropna()

# clean up special characters
df_deg["sentence"] = df_deg["sentence"].replace(r'\n',' ',regex=True)
df_deg["sentence"] = df_deg["sentence"].replace(r'\t',' ',regex=True)
df_deg["sentence"] = df_deg["sentence"].replace(r'\r',' ',regex=True)
df_deg["label"] = df_deg["label"].replace(r'\r','',regex=True)
df_deg["label"] = df_deg["label"].replace(r' ','',regex=True)

# df_midnight = df_midnight[df_midnight['sentence'] != 'sentence']    #remove all rows containing "sentence"
assert df_deg["sentence"].isnull().any() == False, "there were NaNs in the sentence column, ERROR"
assert df_deg["label"].isnull().any() == False, "there were NaNs in the score column, ERROR"

num_degrees=8   # Humour degree has 0-5 possibilities
subdir_name='../data-training/complete/'

train, dev = train_test_split(df_deg, test_size=0.2, random_state=42)
dev, test = train_test_split(dev, test_size=0.5, random_state=42)
print("writing train, dev, and test with shapes", train.shape, dev.shape, test.shape)
suffix="_htype_balanced_0to" + str(num_degrees)
train.to_csv(os.path.join("..", subdir_name, "train"+suffix+".tsv"), sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")
dev.to_csv(os.path.join("..", subdir_name, "dev"+suffix+".tsv"),  sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")
test.to_csv(os.path.join("..", subdir_name, "test"+suffix+".tsv"),  sep="\t",  escapechar='\\', index=None, header=['sentence','label'], encoding="UTF-8")

writing train, dev, and test with shapes (25371, 2) (3171, 2) (3172, 2)
